In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from grid import generate_lats_lons
from paths import path_samoc, path_CESM, grid_file, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from paths import path_ocn_ctrl, path_ocn_ctrl_rect, path_ocn_rcp, path_ocn_rcp_rect
from paths import file_RMASK_ocn, file_RMASK_ocn_rect, file_RMASK_ocn_low
from constants import imt, jmt, km

## make region mask xr dataarrays

In [ ]:
RMASK_ocn = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK
RMASK_ocn = RMASK_ocn.drop(['ULONG', 'ULAT'])
RMASK_ocn.to_netcdf(file_RMASK_ocn)

In [ ]:
RMASK_ocn_low = xr.open_dataset(file_ex_ocn_lpd, decode_times=False).REGION_MASK
RMASK_ocn_low = RMASK_ocn_low.drop(['ULONG', 'ULAT'])
RMASK_ocn_low.to_netcdf(file_RMASK_ocn_low)

RMASK_ocn_rect is generated in `regrid.ipynb`

## latitudes/longitudes from binary file

In [ ]:
lats,lons  = generate_lats_lons('ocn')
f, ax = plt.subplots(1,2, sharey=True)
ax[0].imshow(lats)
ax[1].imshow(lons)

In [ ]:
from xr_DataArrays import xr_DZ, xr_AREA
DZT = xr_DZ('ocn')
TAREA = xr_AREA('ocn')
print(f'The volument of the ocean is {((TAREA*DZT).sum().item()/1e9):.2e} km^3.')
print(f'The surface of the ocean is {(TAREA.sum()/1e6).item():.2e} km^2.')

In [ ]:
plt.imshow(DZT[:,1000,:], aspect='auto')
plt.colorbar()

In [ ]:
plt.imshow(TAREA, cmap='viridis', origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.SouthPolarStereo())
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea(central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
plt.figure(figsize=(5.915, 3))
ax = plt.axes(projection=ccrs.NearsidePerspective(satellite_height=1e7, central_latitude=-90))
ax.coastlines(resolution='110m')
ax.gridlines()

In [ ]:
from analysis.paths import file_ex_ocn_hires

In [ ]:
C = xr.open_dataset(file_ex_ocn_hires, decode_times=False)

In [ ]:
plt.imshow(C.KMT, origin='lowerleft')
plt.colorbar()
plt.tight_layout()

In [ ]:
from read_binary import read_binary_2D_double
from paths import file_geometry
pbc = read_binary_2D_double(file_geometry, 3600, 2400, 1)  # [lon, lat]
plt.imshow(pbc[500:1000, :500].T, origin='lowerleft')
plt.colorbar()

In [ ]:
DZT = generate_xr_DZ('ocn')
DZT_int = DZT.sum(dim='z_t')

In [ ]:
DZT_int.plot()
plt.figure()
DZT_int[:500, 500:1000].plot()

# Regions

In [ ]:
plt.imshow(C.REGION_MASK, cmap='tab20', origin='lowerleft', vmin=-1.1, vmax=18)
plt.colorbar()
plt.tight_layout()

In [ ]:
np.unique(C.REGION_MASK.values)

In [ ]:
from regions import regions
print(regions)

In [ ]:
%%time
for i, key in enumerate(regions):
    surf = xr_surf_int(D[0,:,:], TAREA.where(C.REGION_MASK==key)     )/1e6
    vol  = xr_vol_int( D       , TAREA.where(C.REGION_MASK==key), DZT)/1e9
    print(f'{i:2} {key:4} {regions[key]:16}: {surf:10.2e} km^2, {vol:10.2e} km^3')